In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Import packages & Load data

In [2]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple
import org.jetbrains.letsPlot.intern.Plot
import org.um.feri.ears.statistic.rating_system.true_skill.TrueSkillRating
import org.um.feri.ears.statistic.rating_system.true_skill.Team
import org.um.feri.ears.statistic.rating_system.GameInfo
import org.um.feri.ears.statistic.rating_system.Player
import org.um.feri.ears.statistic.rating_system.true_skill.TwoPlayerTrueSkillCalculator

In [3]:
var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData: GPAlgorithmMultiConfigurationsProgressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

Progress data loaded from file: org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData@46c7cf17
Data size: 3


In [4]:
val gpDataFile = """C:\Users\marko\Documents\FERI\Projekti\EARS\GPAlgorithmExecutor.ser""";
var gpAlgorithmExecutor: GPAlgorithmExecutor = GPAlgorithmExecutor.deserializeGPAlgorithmExecutorState(gpDataFile);

# Prepare helper classes & functions

In [5]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [6]:
class MethodScore(
    var methodName: String,
    var wins: Int,
    var draws: Int,
    var losses: Int,
    var player: Player
) {
    constructor(methodName: String) : this(methodName, 0, 0, 0, Player(methodName))
}

In [7]:
class MethodRating (
    var methodName: String,
    var rating: Double,
    var stdDeviation: Double
) {
    constructor(methodName: String) : this(methodName, 0.0, 0.0)
}


In [8]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult, sumAbsValues: Boolean = false): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        if(sumAbsValues){
            sum += Math.abs(individualMatchResult.individualValues[key]!!)
        }
        else {
            sum += individualMatchResult.individualValues[key]!!
        }
    }

    return sum
}

In [9]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, avgGenIndividualRatings: MutableList<Double>?, bestGenIndividualsRatingAvgFitness: MutableList<Double>?, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, avgGenIndividualAvgFitnesses: MutableList<Double>?){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.population[0]
        var worstIndividualRating = genProgressData.population[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.population[0]
        var worstIndividualAvgFitness = genProgressData.population[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        if (bestGenIndividualsRating != null) {
            bestGenIndividualsRating.add(bestGenIndividualRating)
        }
        if (worstGenIndividualsRating != null) {
            worstGenIndividualsRating.add(worstIndividualRating)
        }

        avgGenIndividualRating /= genProgressData.population.size
        if (avgGenIndividualRatings != null) {
            avgGenIndividualRatings.add(avgGenIndividualRating)
        }

        if(bestGenIndividualsRatingAvgFitness != null){
            bestGenIndividualsRatingAvgFitness.add(sumIndividualValues(bestGenIndividualRating.finalIndividualFitness.avgMatchResult))
        }

        if (bestGenIndividualAvgFitnesses != null) {
            bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        }
        if (worstGenIndividualAvgFitnesses != null) {
            worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)
        }

        avgGenIndividualAvgFitness /= genProgressData.population.size
        if (avgGenIndividualAvgFitnesses != null) {
            avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
        }
    }
}

In [10]:
// TODO Move this to GPProgramSolutionSimple
fun calculateDiversity(solution1: GPProgramSolutionSimple, solution2: GPProgramSolutionSimple, w1: Double, w2: Double) : Double {
    var nodeCountDiversity = 0.0

    // Gather all different node count key
    val nodeCountKeys = mutableSetOf<String>()
    nodeCountKeys.addAll(solution1.nodeCounts.keys)
    nodeCountKeys.addAll(solution2.nodeCounts.keys)

    // Calculate node count diversity
    for (key in nodeCountKeys) {
        val nodeCount1 = solution1.nodeCounts[key] ?: 0
        val nodeCount2 = solution2.nodeCounts[key] ?: 0
        nodeCountDiversity += Math.abs(nodeCount1 - nodeCount2)
    }

    // Normalize node count diversity
    nodeCountDiversity /= (solution1.treeSize + solution2.treeSize)

    // Calculate avg fitness diversity
    var avgFitnessDiversity = 0.0

    // Gather all different avg fitness individual values key
    val avgFitnessKeys = mutableSetOf<String>()
    avgFitnessKeys.addAll(solution1.finalIndividualFitness.avgMatchResult.individualValues.keys)
    avgFitnessKeys.addAll(solution2.finalIndividualFitness.avgMatchResult.individualValues.keys)

    // Calculate avg fitness diversity
    for (key in avgFitnessKeys) {
        val avgFitness1 = solution1.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0
        val avgFitness2 = solution2.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0

        avgFitnessDiversity += Math.abs(avgFitness1 - avgFitness2)
    }

    // Normalize avg fitness diversity
    avgFitnessDiversity /= (Math.abs(sumIndividualValues(solution1.finalIndividualFitness.avgMatchResult, true)) + Math.abs(sumIndividualValues(solution2.finalIndividualFitness.avgMatchResult, true)))

    // Calculate final diversity
    return w1 * avgFitnessDiversity + w2 * nodeCountDiversity
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [11]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
// Input params for visualization END

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd){
    for (selectedRun in selectedRunStart..selectedRunEnd){
        // Prepare data
        val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
        val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
        val avgGenIndividualRatings = mutableListOf<Double>()
        val bestGenIndividualsRatingAvgFitness = mutableListOf<Double>()

        val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
        val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
        val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

        prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualsRatingAvgFitness, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

        var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
        var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

        var lastPhaseName = "";
        val generations = mutableListOf<String>()
        for (genProgressData in multiRunProgressData.gensProgressData) {
            generations.add(genProgressData.generation.toString() + genProgressData.executionPhaseName.substring(genProgressData.executionPhaseName.length - 6, genProgressData.executionPhaseName.length))
        }

        // Display data
        val dfRating = dataFrameOf(
            "Generation" to generations + generations + generations,
            "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
            "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
        )

        val dfRatingAvgFitness = dataFrameOf(
            "Generation" to generations,
            "AvgFitness" to bestGenIndividualsRatingAvgFitness,
            "Individual" to List(generations.size) { "Average" }
        )

        val dfAvgFitness = dataFrameOf(
            "Generation" to generations + generations + generations,
            "AvgFitness" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
            "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
        )

        // The plot variable stores a plot with the temperature values for each city
        val linePlotRating =
            letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Gen Rating (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        val linePlotRatingAvgFitness =
            letsPlot(dfRatingAvgFitness.toMap()) { x = "Generation"; y = "AvgFitness"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Gen Rating - Avg fitness (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        val linePlotAvgFitness =
            letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "AvgFitness"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Gen Avg fitness (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        val plots = listOf(linePlotRating, linePlotRatingAvgFitness, linePlotAvgFitness)
        //val plots = listOf(linePlotRating, null, linePlotRatingAvgFitness, null, linePlotAvgFitness, null)
        gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()
        //gggrid(plots, ncol=2, widths = listOf(1,0)).show()
    }
}

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [ ]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 1, 0)) // Individual at position

val maxValues = mutableMapOf<String, Double>()
maxValues["SectorExploration"] = 5.0
maxValues["PowerUp_Pickup_Health"] = 10.0
maxValues["PowerUp_Pickup_Ammo"] = 15.0
maxValues["PowerUp_Pickup_Shield"] = 5.0
maxValues["MissilesFired"] = 20.0
maxValues["MissilesFiredAccuracy"] = 50.0
maxValues["SurvivalBonus"] = 5.0
maxValues["OpponentTrackingBonus"] = 5.0
maxValues["OpponentDestroyedBonus"] = 500.0
maxValues["DamageTakenPenalty"] = 50.0
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, null, null, null, null, null)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

// Find all fitness keys
val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            System.out.println(key + ": 0.0")
            continue
        }
        dataset.add(Math.abs(individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!) / maxValues[key]!!)
        System.out.println(key + ": " + individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    System.out.println("/////////////////////////////")
    datasets.add(dataset)
}

var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

# Selected Individual Details

In [ ]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 1, 0)) // Individual at position
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, null, null, null, null, null)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 800 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)

    System.out.println(
        "Individual: " + "(Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId + ")\nChangesCount: " + individual.changesCount + "\nFunctionNodesCount: " + individual.functionNodes + "\nTerminalNodesCount: " + individual.terminalNodes + "\nTreeSize: " + individual.treeSize + "\nTreeDepth: " + individual.treeDepth + "\nFitness: " + (individual.finalIndividualFitness.additionalValues["Rating"].toString() + ", " + individual.finalIndividualFitness.additionalValues["StdDeviation"]) + "\nIndividualAvgFitness: " + sumIndividualValues(individual.finalIndividualFitness.avgMatchResult) + "\nIndividualAvgFitnessValues: " + individual.finalIndividualFitness.avgMatchResult.individualValues.map { it.key + ": " + it.value }.toString()
        + "\n\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"
    )

    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

# Genetic Diversity Graph
Genetic diversity of the population based on the avg fitness and node counts

### Calculation of genetic diversity between 2 individuals
1. Difference in avg fitness -> df
2. Difference in node counts -> dn
3. Normalize both values and multiply by the weights -> diversity = w1 * df_norm +  w2 * dn_norm


In [12]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
val w1 = 0.5
val w2 = 0.5
// Input params for visualization END

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    for (selectedRun in selectedRunStart..selectedRunEnd) {
        val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

        prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, null, null, null, null, null, null)

        val gpAlgorithmRunGeneticDiversity = mutableListOf<List<Double?>>()

        for(i in 0 until bestGenIndividualsRating.size) {
            var gpAlgorithmGenGeneticDiversity = mutableListOf<Double?>()
            for (j in 0 until bestGenIndividualsRating.size) {
                if(j < i){
                    var diversity = calculateDiversity(bestGenIndividualsRating[i], bestGenIndividualsRating[j], w1, w2)
                    if(diversity > 1.0 || diversity < 0.0) {
                        throw Exception("Diversity out of bounds: " + diversity + ", i: " + i + ", j:" + j)
                    }
                    gpAlgorithmGenGeneticDiversity.add(diversity)
                }
                else {
                    gpAlgorithmGenGeneticDiversity.add(null)
                }
            }
            gpAlgorithmRunGeneticDiversity.add(gpAlgorithmGenGeneticDiversity)
        }

        var individuals = mutableListOf<Int>()
        val generations = mutableListOf<String>()

        for (i in 0 until gpAlgorithmRunGeneticDiversity.size) {
            generations.addAll(List(gpAlgorithmRunGeneticDiversity[i].size){(i + 1).toString()})

            for (i in 0 until gpAlgorithmRunGeneticDiversity[i].size) {
                individuals.add(i + 1)
            }
        }

        val df = dataFrameOf(
            "individuals" to individuals,
            "generations" to generations,
            "diversity" to gpAlgorithmRunGeneticDiversity.flatten()
        )

        val plot = letsPlot(df.toMap()) { x = "individuals"; y = "generations"; fill = "diversity" } + geomTile() + ggtitle("Genetic Diversity (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")
        plots.add(plot)
    }
    if(plots.size % 3 > 0){
        plots.addAll(List(3 - plots.size % 3){null})
    }
}

gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()

# Master Tournament Graph
Tournament between generation best individuals

In [13]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
// Input params for visualization END

val n = 3

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    for (selectedRun in selectedRunStart..selectedRunEnd) {
        val Generation = mutableListOf<Int>()
        val Rating = mutableListOf<Double>()

        val masterTournamentGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].masterTournamentGraphData;
        for (i in 0 until masterTournamentGraphData.size) {
            Generation.addAll(List(n) {i + 1})
            val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
            Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
        }

        val data = mapOf<String, Any>(
            "Generation" to Generation,
            "Rating" to Rating
        )

        val plot = letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Master Tournament (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")

        plots.add(plot)
    }
    if(plots.size % 3 > 0){
        plots.addAll(List(3 - plots.size % 3){null})
    }
}

gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()

# Convergence graph
Best individual in the final generation is matched with every best individual from previous generations

In [14]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
val selectedRunStart = 0
val selectedRunEnd = 2
// Input params for visualization END

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    for (selectedRun in selectedRunStart..selectedRunEnd) {
        val convergenceGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;

        var generations = mutableListOf<String>()
        var fitnesses = mutableListOf<Float>()

        var lastOpponentMatchIndex: Int = -1;
        for (individualMatch in convergenceGraphData.finalIndividualFitness.individualMatchResults){
            // If generations already contains individualMatch.opponentsIDs[0] then just sum fitness
            lastOpponentMatchIndex = generations.indexOf(individualMatch.opponentsIDs[0].toString());

            // Individual already played against this individual -> just sum fitness
            if(lastOpponentMatchIndex > -1){
                fitnesses[lastOpponentMatchIndex] += individualMatch.value;
            }
            else {
                generations.add(individualMatch.opponentsIDs[0].toString())
                fitnesses.add(individualMatch.value)
            }
        }

        // Display data
        val df = dataFrameOf(
            "Generation" to generations,
            "Fitness" to fitnesses
        )

        val plot = letsPlot(df.toMap()) { x = "Generation"; y = "Fitness"} + geomLine() + geomPoint(shape = 20) + ggtitle("Convergence Graph (Conf: " + selectedConfiguration + ", Run: " + selectedRun + ")")
        plots.add(plot)
    }
    if(plots.size % 3 > 0){
        plots.addAll(List(3 - plots.size % 3){null})
    }
}

gggrid(plots, ncol=3, widths = listOf(1,1,1)).show()

# Grand Master Tournament
1. Master tournament between bestRunSolutions (final best gen solution from each run) -> For each method individually

In [15]:
// Input params for visualization START
val selectedConfigurationStart = 0
val selectedConfigurationEnd = 2
// Input params for visualization END

val n = 3

val plots = mutableListOf<Plot?>()

for(selectedConfiguration in selectedConfigurationStart..selectedConfigurationEnd) {
    val IndividualIDs = mutableListOf<Int>()
    val Rating = mutableListOf<Double>()

    val masterTournamentGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].masterMasterTournamentGraphData;
    for (i in 0 until masterTournamentGraphData.size) {
        IndividualIDs.addAll(List(n) { i })
        val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
        Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
    }

    val data = mapOf<String, Any>(
        "IndividualID" to IndividualIDs,
        "Rating" to Rating
    )

    val plot = letsPlot(data) { y = "IndividualID"; x = "Rating"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Master Tournament (Conf: " + selectedConfiguration + ")")

    plots.add(plot)
    plots.add(null)
}

gggrid(plots, ncol=2, widths = listOf(1.5, 0)).show()

# Final Master Tournament
1. Master tournament between bestRunSolutions (final best gen solution from each run - they each represent one individual on graph) for all methods.

In [16]:
val n = 3

val IndividualIDs = mutableListOf<Long>()
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

val masterTournamentGraphData = progressData.finalMasterTournamentGraphData;
for (i in 0 until masterTournamentGraphData.size) {
    IndividualIDs.addAll(List(n){ masterTournamentGraphData[i].individualId })
    val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))

    Method.addAll(List(n){ masterTournamentGraphData[i].configurationName })
}

val data = dataFrameOf(
    "IndividualID" to IndividualIDs,
    "Rating" to Rating,
    "Method" to Method
)

val plot = letsPlot(data.toMap()) { y = "IndividualID"; x = "Rating"; color = "Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament") + ggtb()

plot.show()

# Final Master Tournament (Grouped) - Method 1

In [17]:
val methodScores = mutableListOf<MethodScore>()

val masterTournamentGraphData = progressData.finalMasterTournamentGraphData;

// Initialize method scores
for(config in gpAlgorithmExecutor.configuration.Configurations){
    methodScores.add(MethodScore(config.Name))
}

val calculator: TwoPlayerTrueSkillCalculator = TwoPlayerTrueSkillCalculator() // TODO Should we use TwoTeamTrueSkillCalculator instead?
val gameInfo: GameInfo = GameInfo.getDefaultGameInfo()

var playerIndexes = List(masterTournamentGraphData.size){ it }
playerIndexes = playerIndexes.shuffled()

for(i in 0 until masterTournamentGraphData.size){
    for(j in i until masterTournamentGraphData.size) {
        if(i == j) {
            continue
        }

        val individual1 = masterTournamentGraphData[i]
        val individual2 = masterTournamentGraphData[j]

        if(individual1.configurationName == individual2.configurationName){
            continue
        }

        val individual1MethodScore = methodScores.find { it.methodName == individual1.configurationName }!!
        val individual2MethodScore = methodScores.find { it.methodName == individual2.configurationName }!!

        val individual1Score = individual1.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual2.individualId.toInt() }.map { it.value }.sum()
        val individual2Score = individual2.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual1.individualId.toInt() }.map { it.value }.sum()

        // Determine match result & set scores
        var scores: List<Int>
        if(individual1Score < individual2Score){
            individual1MethodScore.wins++
            individual2MethodScore.losses++
            scores = listOf(1,2)
        }
        else if(individual1Score > individual2Score) {
            individual1MethodScore.losses++
            individual2MethodScore.wins++
            scores = listOf(2, 1)
        }
        else {
            individual1MethodScore.draws++
            individual2MethodScore.draws++
            scores = listOf(1,1)
        }

        // Update player ratings
        val team1 = Team(individual1MethodScore.player, TrueSkillRating(individual1MethodScore.player.freeForAllTrueSkill.rating, individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation))

        val team2 = Team(individual2MethodScore!!.player, TrueSkillRating(individual2MethodScore.player.freeForAllTrueSkill.rating, individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation))

        val teams = Team.concat(team1, team2)

        val newRatings = calculator.calculateNewRatings(gameInfo, teams, scores.get(0), scores.get(1))

        val player1NewRating = newRatings[individual1MethodScore.player]
        val player2NewRating = newRatings[individual2MethodScore.player]

        individual1MethodScore.player.freeForAllTrueSkill.rating =  player1NewRating!!.rating
        individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation =  player1NewRating.ratingDeviation

        individual2MethodScore.player.freeForAllTrueSkill.rating =  player2NewRating!!.rating
        individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation = player2NewRating.ratingDeviation
    }
}

val dataTemp = dataFrameOf(
    "Method" to methodScores.map { it.methodName },
    "Wins" to methodScores.map { it.wins },
    "Draws" to methodScores.map { it.draws },
    "Losses" to methodScores.map { it.losses }
)

System.out.println(dataTemp)

val n = 3
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

for(i in 0 until methodScores.size){
    Method.addAll(List(n){ methodScores.get(i).methodName })

    Rating.addAll(listOf(methodScores.get(i).player.freeForAllTrueSkill.rating - (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation), methodScores.get(i).player.freeForAllTrueSkill.rating, methodScores.get(i).player.freeForAllTrueSkill.rating + (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation)))
}

val data = dataFrameOf(
    "Method" to Method,
    "Rating" to Rating,
)

val plot = letsPlot(data.toMap()) { y = "Method"; x = "Rating"; color="Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament (Grouped)") + ggtb()

plot.show()

            Method Wins Draws Losses
 0 MethodTrueSkill    2     4     15
 1   MethodGlicko2   20     0      4
 2       MethodElo    7     4     10



# Final Master Tournament (Grouped) - Method 2

In [18]:
val masterTournamentGraphData = progressData.finalMasterTournamentGraphData;

var methodRatings = mutableListOf<MethodRating>()

// Initialize method raings
for(config in gpAlgorithmExecutor.configuration.Configurations){
    methodRatings.add(MethodRating(config.Name))
}

for(mmethodRating in methodRatings){
    // find all individuals from masterTournamentGraphData with the same method name
    val individuals = masterTournamentGraphData.filter { it.configurationName == mmethodRating.methodName }

    // Calculate group rating and std deviation
    var groupRating = 0.0
    var groupStdDeviation = 0.0

    for(individual in individuals){
        groupRating += individual.finalIndividualFitness.additionalValues["Rating"]!!
        groupStdDeviation += Math.pow(individual.finalIndividualFitness.additionalValues["StdDeviation"]!!, 2.0)
    }

    groupStdDeviation = Math.sqrt(groupStdDeviation)

    mmethodRating.rating = groupRating + 3 * groupStdDeviation
    mmethodRating.stdDeviation = groupStdDeviation
}

val n = 3
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

for(mmethodRating in methodRatings){
    Method.addAll(List(n){ mmethodRating.methodName })

    val rating = Math.abs(mmethodRating.rating)
    Rating.addAll(listOf(rating - (2 * mmethodRating.stdDeviation), rating, rating + (2 * mmethodRating.stdDeviation)))
}

val data = dataFrameOf(
    "Method" to Method,
    "Rating" to Rating,
)

val plot = letsPlot(data.toMap()) { y = "Method"; x = "Rating"; color="Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament (Grouped)") + ggtb()

plot.show()

# Final Master Tournament (Grouped) - Method 3
Only compare individuals from the same run

In [19]:
import kotlin.random.Random

val finalMasterTournamentGraphData = progressData.finalMasterTournamentGraphData;

val playerGroups = mutableMapOf<String, MutableList<GPProgramSolutionSimple>>()

val methodScores = mutableListOf<MethodScore>()

// Initialize method scores
for(config in gpAlgorithmExecutor.configuration.Configurations){
    methodScores.add(MethodScore(config.Name))
}

for(individual in finalMasterTournamentGraphData){
    if(playerGroups.containsKey(individual.configurationName)){
        playerGroups[individual.configurationName]?.add(individual)
    }
    else {
        playerGroups[individual.configurationName] = mutableListOf(individual)
    }
}

val calculator: TwoPlayerTrueSkillCalculator = TwoPlayerTrueSkillCalculator() // TODO Should we use TwoTeamTrueSkillCalculator instead?
val gameInfo: GameInfo = GameInfo.getDefaultGameInfo()

var playerIndexes = List(playerGroups[methodScores.get(0).methodName]!!.size) {it}

val random = Random(123)
playerIndexes = playerIndexes.shuffled(random)

for(playerIndex in playerIndexes) {
    var playerGroupKeys1 = playerGroups.keys.toList()
    playerGroupKeys1 = playerGroupKeys1.shuffled()

    var playerGroupKeys2 = playerGroups.keys.toList()
    playerGroupKeys2 = playerGroupKeys2.shuffled()

    for(playerGroupKey1 in playerGroupKeys1) {
        for (playerGroupKey2 in playerGroupKeys2) {
            if (playerGroupKey1 == playerGroupKey2) {
                continue
            }

            val playerGroup1 = playerGroups[playerGroupKey1]!!
            val playerGroup2 = playerGroups[playerGroupKey2]!!

            val individual1 = playerGroup1[playerIndex]
            val individual2 = playerGroup2[playerIndex]

            // Get match results
            val individual1Score =
                individual1.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual2.individualId.toInt() }
                    .map { it.value }.sum()
            val individual2Score =
                individual2.finalIndividualFitness.individualMatchResults.filter { it.opponentsIDs[0] == individual1.individualId.toInt() }
                    .map { it.value }.sum()

            val individual1MethodScore = methodScores.find { it.methodName == individual1.configurationName }!!
            val individual2MethodScore = methodScores.find { it.methodName == individual2.configurationName }

            //System.out.println(individual1.configurationName + "(" + individual1.individualId + ") vs " + individual2.configurationName  + "(" + individual2.individualId + ") -> " + individual1Score + ": " + individual2Score)

            // Determine match result & set scores
            var scores: List<Int>
            if (individual1Score < individual2Score) {
                individual1MethodScore.wins++
                scores = listOf(1, 2)
            } else if (individual1Score > individual2Score) {
                individual1MethodScore.losses++
                scores = listOf(2, 1)
            } else {
                individual1MethodScore.draws++
                scores = listOf(1, 1)
            }

            // Update player ratings
            val team1 = Team(
                individual1MethodScore.player,
                TrueSkillRating(
                    individual1MethodScore.player.freeForAllTrueSkill.rating,
                    individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation
                )
            )

            val team2 = Team(
                individual2MethodScore!!.player,
                TrueSkillRating(
                    individual2MethodScore.player.freeForAllTrueSkill.rating,
                    individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation
                )
            )

            val teams = Team.concat(team1, team2)

            val newRatings = calculator.calculateNewRatings(gameInfo, teams, scores.get(0), scores.get(1))

            val player1NewRating = newRatings[individual1MethodScore.player]
            val player2NewRating = newRatings[individual2MethodScore.player]

            individual1MethodScore.player.freeForAllTrueSkill.rating = player1NewRating!!.rating
            individual1MethodScore.player.freeForAllTrueSkill.ratingDeviation = player1NewRating.ratingDeviation

            // Only update individual1MethodScore.player !!!
            //individual2MethodScore.player.freeForAllTrueSkill.rating =  player2NewRating!!.rating
            //individual2MethodScore.player.freeForAllTrueSkill.ratingDeviation = player2NewRating.ratingDeviation
        }
    }
}

// Temp display
val dataTemp = dataFrameOf(
    "Method" to methodScores.map { it.methodName },
    "Wins" to methodScores.map { it.wins },
    "Draws" to methodScores.map { it.draws },
    "Losses" to methodScores.map { it.losses }
)

System.out.println(dataTemp)

val n = 3
val Rating = mutableListOf<Double>()
val Method = mutableListOf<String>()

for(i in 0 until methodScores.size){
    Method.addAll(List(n){ methodScores.get(i).methodName })

    Rating.addAll(listOf(methodScores.get(i).player.freeForAllTrueSkill.rating - (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation), methodScores.get(i).player.freeForAllTrueSkill.rating, methodScores.get(i).player.freeForAllTrueSkill.rating + (2 * methodScores.get(i).player.freeForAllTrueSkill.ratingDeviation)))
}

val data = dataFrameOf(
    "Method" to Method,
    "Rating" to Rating,
)

val plot = letsPlot(data.toMap()) { y = "Method"; x = "Rating"; color="Method"}  + geomBoxplot(whiskerWidth = 0.2, orientation="y") + ggtitle("Final Master Tournament (Grouped)") + ggtb()

plot.show()

            Method Wins Draws Losses
 0 MethodTrueSkill    1     1      4


 1   MethodGlicko2    5     0      1
 2       MethodElo    2     1      3

